## WOrking with Azure Machine Learning without MLflow

In [ ]:
# pip install azure-ai-ml

#### Working with workspace and client

In [29]:
# details
subscritpion_id="53eb2592-79d7-4d14-a92a-b97966ccf1c9"
resource_group="bdc"
workspace="ey2025"

In [30]:
# authenticate
from azure.identity import InteractiveBrowserCredential
credential = InteractiveBrowserCredential(tenant_id="13a86542-2185-4187-8e07-7512f5525c55")


In [31]:
# create ML client
from azure.ai.ml import MLClient
ml_client = MLClient(credential=credential,subscription_id=subscritpion_id,
                     resource_group_name=resource_group,workspace_name=workspace)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [32]:
# access the workspace
from azure.ai.ml.entities import Workspace
try:
    ws = ml_client.workspace.get(workspace)
    print("Workspace already exists")
except:
    ws = Workspace(name=workspace,location='centralindia')
    ws = ml_client.workspaces.begin_create(ws)

### Creating a compute

In [33]:
from azure.ai.ml.entities import ComputeInstance

name = 'mlcompute26-anshu'

try:
    amlcompute = ml_client.compute.get(name)
    print(f"you already have a compute named {name} we will reuse it")
except:
    print('creating a new compute target')
    amlcompute = ComputeInstance(
        name=name,
        type='amlcompute',
        size='STANDARD_DS11_V2',
        idle_time_before_shutdown=120,
        
    )

    amlcompute = ml_client.compute.begin_create_or_update(amlcompute)

you already have a compute named mlcompute26-anshu we will reuse it


In [34]:
import os
os.makedirs("AzureML",exist_ok=True)

!wget -q -P AzureML https://raw.githubusercontent.com/MicrosoftLearning/mslearn-mlops/refs/heads/main/production/data/diabetes-prod.csv


In [35]:
%%writefile AzureML/diabetes-training.py

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes-prod.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)



# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

import os
import joblib

os.makedirs('outputs', exist_ok=True)
#file = open("outputs/version.txt","w")
#file.write(str(joblib.__version__))
#file.close()
joblib.dump(model, open("outputs/model.pkl","wb"))

Overwriting AzureML/diabetes-training.py


### Run a job on a compute in Azure Machine Learning

In [36]:
for env in ml_client.environments.list():
    if 'sklearn' in env.name:
        print(env)

creation_context:
  created_at: '2021-12-15T02:18:23.826902+00:00'
  created_by: Microsoft
  created_by_type: User
  last_modified_at: '2021-12-15T02:18:23.826902+00:00'
  last_modified_by: Microsoft
  last_modified_by_type: User
id: azureml:/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/providers/Microsoft.MachineLearningServices/workspaces/ey2025/environments/AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
latest_version: '36'
name: AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
tags: {}

creation_context:
  created_at: '2021-07-14T00:48:00.493289+00:00'
  created_by: Microsoft
  created_by_type: User
  last_modified_at: '2021-07-14T00:48:00.493289+00:00'
  last_modified_by: Microsoft
  last_modified_by_type: User
id: azureml:/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/providers/Microsoft.MachineLearningServices/workspaces/ey2025/environments/AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
latest_version: '49'
name: AzureML-sklearn-0.24-ubuntu18.04-py3

In [37]:
from azure.ai.ml import command

job = command(code="./AzureML",
              command="python diabetes-training.py",
              environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
              compute='mlcompute26-anshu',
              display_name='diabetes-train',
              experiment_name='diabetes-training')

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("monitor your job at ",aml_url)

Uploading AzureML (1.04 MBs): 100%|██████████| 1036853/1036853 [00:00<00:00, 1719387.08it/s]




monitor your job at  https://ml.azure.com/runs/placid_band_qzpj48krrh?wsid=/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourcegroups/bdc/workspaces/ey2025&tid=13a86542-2185-4187-8e07-7512f5525c55


## Register Model

In [38]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

path = f"azureml://jobs/{returned_job.name}/outputs/artifacts/outputs"
file_model = Model(path=path,type=AssetTypes.CUSTOM_MODEL,
                   name='diabetes_model')

out = ml_client.models.create_or_update(file_model)
out.path

'azureml://subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/workspaces/ey2025/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.placid_band_qzpj48krrh/outputs'

# Deploy the model

In [40]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

import datetime
ep_name = 'diabetes-endpoint'
ep = ManagedOnlineEndpoint(name=ep_name,auth_mode="key")

ep_result = ml_client.online_endpoints.begin_create_or_update(ep).result()

In [41]:
ep = ml_client.online_endpoints.get(name=ep_name)

In [42]:
os.makedirs("AzureML/score", exist_ok=True)

In [43]:
%%writefile AzureML/score/score.py

import os
import joblib
import json 
import logging
import numpy
def init():
    global model
    path = os.path.join(os.getenv("AZUREML_MODEL_DIR"),"outputs/model.pkl") # AZUREML_MODEL_DIR = ./azureml-models/MODEL_NAME/VERSION
    #logging.info(path)
    model = joblib.load(path)
    logging.info("initialization completed")

def run(raw_data):
    logging.info("model: request received")
    data = json.loads(raw_data)["data"]
    data = numpy.array(data)
    result = model.predict(data)
    logging.info("request procsesed")
    return result.tolist()

Writing AzureML/score/score.py


In [45]:
%%writefile AzureML/score/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.0
  - pip=21.3.1
  - scikit-learn=1.0.0
  - scipy=1.7.1
  - numpy
  - pandas==1.3.0
  - pip:
    - azureml-defaults==1.51.0
    - applicationinsights
    - inference-schema[numpy-support]==1.5
    - joblib==1.2.0

Writing AzureML/score/conda.yml


In [48]:
# deployment configurations
from azure.ai.ml.entities import Model,ManagedOnlineDeployment, CodeConfiguration
from azure.ai.ml.constants import AssetTypes

# configure the environment
from azure.ai.ml.entities import Environment
env = Environment(conda_file="AzureML/score/conda.yml",
                  image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230608.v1")

# configure an inference pipeline with a scoring script
code_config = CodeConfiguration(code="AzureML/score",
                                scoring_script='score.py')

model = ml_client.models.get(name=out.name, version=out.version)

# define the online deployment
blue_deployment = ManagedOnlineDeployment(name='blue',
                                          endpoint_name=ep_name,
                                          model=model,
                                          instance_type="STANDARD_F4s_v2",
                                          instance_count=1,
                                          environment=env,
                                          code_configuration=code_config)

In [49]:
blue_deploy_result = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint diabetes-endpoint exists
Uploading score (0.0 MBs): 100%|██████████| 858/858 [00:00<00:00, 2552.07it/s]




.....................................................................................................................................................................................................................

In [50]:
# blue deployment taking 100% traffic
ep.traffic = {"blue":100}
ml_client.online_endpoints.begin_create_or_update(ep)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


In [52]:
print(f"Name: {ep.name} \nStatus: {ep.provisioning_state} \nDescription {ep.description}")

Name: diabetes-endpoint 
Status: Succeeded 
Description None


## Testing deployment

In [53]:
os.makedirs("AzureML/testing",exist_ok=True)

In [54]:
ep.scoring_uri

'https://diabetes-endpoint.eastasia.inference.ml.azure.com/score'

In [55]:
ep_cred = ml_client.online_endpoints.get_keys(name=ep_name).primary_key
ep_cred

'G95JXfhdibpQZLsQ5eFeUa6n8OnNv7T8FXHHSgtIHeAM64XhoUe8JQQJ99BCAAAAAAAAAAAAINFRAZMLxbUi'

In [58]:
import json
import requests

url = 'https://diabetes-endpoint.eastasia.inference.ml.azure.com/score'
api_key = 'G95JXfhdibpQZLsQ5eFeUa6n8OnNv7T8FXHHSgtIHeAM64XhoUe8JQQJ99BCAAAAAAAAAAAAINFRAZMLxbUi'
header = {"Content-Type":"application/json", "Authorization":('Bearer '+api_key),
          'azureml-model-deployment':'blue'}

data = {'data':[
 [9,103,78,25,304,29.58219193,1.282869847,43], 
 [7,115,47,52,35,41.51152348,0.079018568,23]
]}


response = requests.post(url,data=json.dumps(data),headers=header)
response.content

b'[1, 0]'